<a href="https://colab.research.google.com/github/higor-gomes93/mestrado_ft084/blob/main/Twitter_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Código para carregar  o arquvo  corona_tweets_463.csv salvo no GoogleDrive.
#Código para carregar os tweet que serão utilizados: 150.000 regsitros do arquivo corona_tweets_463.csv
from google.colab import drive
import os
from IPython.display import clear_output
from IPython.display import clear_output

drive.mount('/content/drive', force_remount=True)
working_directory = 'MyDrive/TrabalhoFT' #@param {type:"string"}
wd="/content/drive/"+working_directory
os.chdir(wd)

dirpath = os.getcwd()
print("current directory is : " + dirpath)


#Carregando o arquivo do googleDrive sem cabeçalho definido no arquivo e 150.000 linhas
import pandas as pd

header_list = ["id", "score"]
df = pd.read_csv('/content/drive/MyDrive/TrabalhoFT/corona_tweets_463.csv',nrows=150000, usecols=[0,1],names=header_list,skip_blank_lines=True)
x = df[['id','score']]#Read two columns
#x.columns = ['id', 'score']
print(x)

y =  x[['id']] #Read a column

print(y)

#Mostrando quantas linhas e colunas temos no DataFrame
y.shape


#Instalando a biblioteca twarc para Hydaratar os Tweets
%pip install twarc
%pip install jsonlines

#Configurando o twarc para ler os tweets
from twarc import Twarc


consumer_key = "TqOMbK44mhFn8P4Rviae3sM5Y" #@param {type:"string"}
consumer_secret = "Qkog3ytEszYMQ531yPfjXupPNPOoh973v1YP1Oe6YZYWNY19Ba" #@param {type:"string"}
access_token = "1408437861624553480-TiIJbrPCWDQVE7MROBFZvJVho0dyLu" #@param {type:"string"}
access_token_secret = "iqyl70jym6MNvRVBh7Kh8qzZYi5T5CmzZHcvBnDeL8KLf" #@param {type:"string"}

t = Twarc(consumer_key, consumer_secret, access_token, access_token_secret)

#Configurando o arquivo de saida

final_tweet_ids_filename = "/content/drive/MyDrive/TrabalhoFT/IDtweets.csv" #@param {type: "string"}
output_filename = "/content/drive/MyDrive/TrabalhoFT/output.csv" #@param {type: "string"}

#apagando o arquivo anterior
file = '/content/drive/MyDrive/TrabalhoFT/output.txt'
if(os.path.exists(file) and os.path.isfile(file)):
  os.remove(file)
  print("file deleted")
else:
  print("file not found")

#apagando o arquivo anterior
if(os.path.exists(output_filename) and os.path.isfile(output_filename)):
  os.remove(output_filename)
  print("file deleted")
else:
  print("file not found")


#Definindo os id Tweets para Hydratar.
y.to_csv('/content/drive/MyDrive/TrabalhoFT/IDtweets.csv', encoding='utf-8', index=False)

for tweet in t.hydrate('/content/drive/MyDrive/TrabalhoFT/IDtweets.csv'):
    print(tweet['full_text'])
    print(tweet['id'])

import jsonlines, json
# Stores hydrated tweets here as jsonl objects
# Contains one json object per line
output_json_filename = output_filename[:output_filename.index(".")] + ".txt"
ids = []
with open(final_tweet_ids_filename, "r") as ids_file:
    ids = ids_file.read().split()
hydrated_tweets = []
ids_to_hydrate = set(ids)


# Looks at the output file for already hydrated tweets
if os.path.isfile(output_json_filename):
    with jsonlines.open(output_json_filename, "r") as reader:
        for i in reader.iter(type=dict, skip_invalid=True):
            # These tweets have already been hydrated. So remove them from ids_to_hydrate
            hydrated_tweets.append(i)
            ids_to_hydrate.remove(i["id_str"])
            print("ARQUIVO")
print("Total IDs: " + str(len(ids)) + ", IDs to hydrate: " + str(len(ids_to_hydrate)))
print("Hydrated: " + str(len(hydrated_tweets)))

count = len(hydrated_tweets)
start_index = count # The index from where tweets haven't been saved to the output_json_file
# Stores hydrated tweets to output_json_file every num_save iterations.
num_save  = 1000

# Now, use twarc and start hydrating
for tweet in t.hydrate(ids_to_hydrate):
    hydrated_tweets.append(tweet)
    count += 1
    # If num_save iterations have passed,
    if (count % num_save) == 0:
        # Open the output file
        # NOTE: Even if the code stops during IO, only tweets from the current iteration are lost.
        # Older tweets are preserved as the file is written in append mode.
        with jsonlines.open(output_json_filename, "a") as writer:
            print("Started IO")
            # Now write the tweets from start_index. The other tweets don't have to be written
            # as they were already written in a previous iteration or run.
            for hydrated_tweet in hydrated_tweets[start_index:]:
                writer.write(hydrated_tweet)
            print("Finished IO")
        print("Saved " + str(count) + " hydrated tweets.")
        # Now, since everything has been written. Reset start_index
        start_index = count
# There might be tweets unwritten in the last iteration if the count is not a multiple of num_tweets.
# In that case, just write out the remainder of tweets.
if count != start_index:
    print("Here with start_index", start_index)
    with jsonlines.open(output_json_filename, "a") as writer:
        for hydrated_tweet in hydrated_tweets[start_index:]:
           writer.write(hydrated_tweet)

# Convert jsonl to csv
import csv, jsonlines
output_json_filename = output_filename[:output_filename.index(".")] + ".txt"
keyset = ["created_at", "id", "id_str", "full_text", "source", "truncated", "in_reply_to_status_id",
          "in_reply_to_status_id_str", "in_reply_to_user_id", "in_reply_to_user_id_str", 
          "in_reply_to_screen_name", "user", "coordinates", "place", "quoted_status_id",
          "quoted_status_id_str", "is_quote_status", "quoted_status", "retweeted_status", 
          "quote_count", "reply_count", "retweet_count", "favorite_count", "entities", 
          "extended_entities", "favorited", "retweeted", "possibly_sensitive", "filter_level", 
          "lang", "matching_rules", "current_user_retweet", "scopes", "withheld_copyright", 
          "withheld_in_countries", "withheld_scope", "geo", "contributors", "display_text_range",
          "quoted_status_permalink"]       
hydrated_tweets = []
with jsonlines.open(output_json_filename, "r") as reader:
    for i in reader.iter(type=dict, skip_invalid=True):
        hydrated_tweets.append(i)
with  open(output_filename, "w+") as output_file:
    d = csv.DictWriter(output_file, keyset)
    d.writeheader()
    d.writerows(hydrated_tweets)

df_tweets = pd.read_csv('/content/drive/MyDrive/TrabalhoFT/output.csv')

# select desired columns
df_tweets = df_tweets[['id', 'full_text']]
df_text_tweets = df_tweets[['full_text']]

print(df_tweets[['full_text']])
m = pd.merge(x, df_tweets, how = 'left', on = 'id')

#write to the file (tab separated)
m.to_csv('Final.csv', sep=',', index=False)

# --------------------------- PRE PROCESSAMENTO ------------------------------
#Começando a parte do pré-processamento

#[Jfranciscon] Definição das funções para preprocessamento
#!pip install google_trans_new
!pip install git+https://github.com/stanfordnlp/stanza.git
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
import re
import tweepy
from tweepy import OAuthHandler
from textblob import TextBlob
import stanza
stanza.download('en')
#from google_trans_new import google_translator
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import numpy as np

nlp = stanza.Pipeline(lang='en')

def clean_tweet(tweet):
        '''
        Função para limpar o tweet removendo links, caracteres especiais
        utilizando regex.
        '''
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

def translate_tweet(tweet):
        '''
        Função para traduzir o tweet portugues para inglês.
        '''
        translator = google_translator()
        tweet_en = translator.translate(tweet, lang_src='pt', lang_tgt='en')
        return tweet_en
    
def stopwords_tweet(tweet):
        '''
        Função para remover stopwords
        '''
        stop_words = set(stopwords.words('english'))
        word_tokens = word_tokenize(tweet)
        
        filtered_sentence = []
        for w in word_tokens:
            if w not in stop_words:
                filtered_sentence.append(w)
                
        return ' '.join(filtered_sentence)
    
def lemma_tweet(tweet):
        lemmatizer = WordNetLemmatizer()
        tweet_lower = tweet.lower()
        # define função para lemmatize cada palavra com sua POS tag
        # POS_TAGGER_FUNCTION : TYPE 1
        def pos_tagger(nltk_tag):
            if nltk_tag.startswith('J'):
                return wordnet.ADJ
            elif nltk_tag.startswith('V'):
                return wordnet.VERB
            elif nltk_tag.startswith('N'):
                return wordnet.NOUN
            elif nltk_tag.startswith('R'):
                return wordnet.ADV
            elif nltk_tag.startswith('A'):
                return wordnet.ADJ
            elif nltk_tag.startswith('S'):
                return wordnet.ADJ_SAT
            else:          
                return None

        pos_tagged = nltk.pos_tag(nltk.word_tokenize(tweet_lower))
        wordnet_tagged = list(map(lambda x: (x[0], pos_tagger(x[1])), pos_tagged))
        
        lemmatized_sentence = [] #cria a lista para armazenar cada palavra na forma basica
        for word, tag in wordnet_tagged:
            if tag is None:
                # se não tem uma tag, armazena a palavra como está
                lemmatized_sentence.append(word)
            else:        
                # caso contrário seleciona a tag e retorna a forma basica
                lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))        
 
        return ' '.join(lemmatized_sentence)    
    
def get_tweet_sentiment(tweet):
        '''
        Função para classificar os sentimentos em positivo/negativo/neutro do tweet coletado
        1. faz a tradução do portugues para ingles usando a nova api do google (prob na antinga)
        2. usando o metodo textblob no tweet traduzido (tweet_en)
        
        '''
        # INICIO das etapas de PREPROCESSAMENTO DO TWEET
        # 1. traduz para inglês
        #tweet_en = translate_tweet(tweet)
        # 2. limpa caracters, hashtags, http, pontuação etc
        tweet_en = clean_tweet(tweet)
        # 3. identifica e remove stopwords (palavras irrelevantes)
        tweet_en = stopwords_tweet(tweet_en)
        # 4. identifica a forma morfologica e converte a palavra na forma básica
        tweet_en = lemma_tweet(tweet_en)
        
        # TEXTBLOB faz a analise de sentimento com o tweet preprocessado
        analysis = TextBlob(tweet_en)
        neutral_thresh = 0.05
        # define o sentimento-->sentiment
        if analysis.sentiment.polarity >= neutral_thresh:
            return 'positivo'
        elif analysis.sentiment.polarity <= -(neutral_thresh):
            return 'negativo'
        else:
            return 'neutro'

def get_tweet_sentiment_STANZA(tweet):
        # A funcao obtem (media) score do sentimento pelo Stanza para cada tweet
        # Menos 1, de modo a trazer a faixa de pontuação de [0,2] a [-1,1]
        def stanza_analyze(Text):
            document = nlp(Text)
            return np.mean([(i.sentiment - 1) for i in document.sentences]) 
               
        analysis = stanza_analyze(tweet)
        
        neutral_thresh = 0.05
        # define o sentimento-->sentiment
        if analysis >= neutral_thresh:
            return 'positivo'
        elif analysis <= -(neutral_thresh):
            return 'negativo'
        else:
            return 'neutro'

def converte_sentimento_original(sentiment):
        neutral_thresh = 0.05
        if sentiment >= neutral_thresh:
            return 'positivo'
        elif sentiment <= -(neutral_thresh):
            return 'negativo'
        else:
            return 'neutro'


df_tweets['sentimento'] = df_tweets['full_text'].map(get_tweet_sentiment)
df_tweets['sentimento_STANZA'] = df_tweets['full_text'].map(get_tweet_sentiment_STANZA)

# pega os tweets positivos da lista de tweets
ptweets = len(df_tweets.loc[df_tweets['sentimento'] == 'positivo'])
# porcentagem dos tweets positivos
print("% tweets positivos TEXTBLOB: {} %".format(100*(ptweets/len(df_tweets))))
# pega os tweets negativos da lista de tweets
ntweets = len(df_tweets.loc[df_tweets['sentimento'] == 'negativo'])
# porcentagem dos tweets negativos
print("% tweets negativos TEXTBLOB: {} %".format(100*(ntweets/len(df_tweets))))
# porcentagem dos tweets neutros
print("% tweets neutros TEXTBLOB: {} % \
    ".format(100*((len(df_tweets) -(ntweets+ptweets))/len(df_tweets))))

# pega os tweets positivos da lista STANZA
ptweets = len(df_tweets.loc[df_tweets['sentimento_STANZA'] == 'positivo'])
# porcentagem dos tweets positivos STANZA
print("% tweets positivos STANZA: {} %".format(100*(ptweets/len(df_tweets))))
# pega os tweets negativos da lista STANZA 
ntweets = len(df_tweets.loc[df_tweets['sentimento_STANZA'] == 'negativo'])
# porcentagem dos tweets negativos STANZA
print("% tweets negativos STANZA: {} %".format(100*(ntweets/len(df_tweets))))
# porcentagem dos tweets neutros STANZA
print("% tweets neutros STANZA: {} % \
    ".format(100*((len(df_tweets) -(ntweets+ptweets))/len(df_tweets))))

df_original = pd.read_csv('/content/drive/MyDrive/TrabalhoFT/Final.csv', delimiter=',', usecols=[0,1])
df_original['sentimento_original'] = df_original['score'].map(converte_sentimento_original)

# pega os tweets positivos da lista de tweets originais
ptweets = len(df_original.loc[df_original['sentimento_original'] == 'positivo'])
# porcentagem dos tweets positivos originais
print("% tweets positivos ORIGINAIS: {} %".format(100*(ptweets/len(df_original))))
# pega os tweets negativos da lista de tweets originais
ntweets = len(df_original.loc[df_original['sentimento_original'] == 'negativo'])
# porcentagem dos tweets negativos originais
print("% tweets negativos ORIGINAIS: {} %".format(100*(ntweets/len(df_original))))
# porcentagem dos tweets neutros originais
print("% tweets neutros ORIGINAIS: {} % \
    ".format(100*((len(df_original) -(ntweets+ptweets))/len(df_original))))



Mounted at /content/drive
current directory is : /content/drive/MyDrive/TrabalhoFT
                         id     score
0       1407917783569031170  0.000000
1       1407917783824801792  0.383333
2       1407917783929655303  0.250000
3       1407917783971684356  0.166667
4       1407917784269529091  0.100000
...                     ...       ...
149995  1407953168932220928  0.910000
149996  1407953169229955077  0.150000
149997  1407953168848211973  0.000000
149998  1407953169418756096 -0.100000
149999  1407953169502674945  0.227273

[150000 rows x 2 columns]
                         id
0       1407917783569031170
1       1407917783824801792
2       1407917783929655303
3       1407917783971684356
4       1407917784269529091
...                     ...
149995  1407953168932220928
149996  1407953169229955077
149997  1407953168848211973
149998  1407953169418756096
149999  1407953169502674945

[150000 rows x 1 columns]
     |████████████████████████████████| 61kB 5.2MB/s 
  Created wheel f

Started IO
Finished IO
Saved 84000 hydrated tweets.
Started IO
Finished IO
Saved 85000 hydrated tweets.
Started IO
Finished IO
Saved 86000 hydrated tweets.
Started IO
Finished IO
Saved 87000 hydrated tweets.
Started IO
Finished IO
Saved 88000 hydrated tweets.
Started IO
Finished IO
Saved 89000 hydrated tweets.
Started IO
Finished IO
Saved 90000 hydrated tweets.
Started IO
Finished IO
Saved 91000 hydrated tweets.
Started IO
Finished IO
Saved 92000 hydrated tweets.
Started IO
Finished IO
Saved 93000 hydrated tweets.
Started IO
Finished IO
Saved 94000 hydrated tweets.
Started IO
Finished IO
Saved 95000 hydrated tweets.
Started IO
Finished IO
Saved 96000 hydrated tweets.
Started IO
Finished IO
Saved 97000 hydrated tweets.
Started IO
Finished IO
Saved 98000 hydrated tweets.
Started IO
Finished IO
Saved 99000 hydrated tweets.
Started IO
Finished IO
Saved 100000 hydrated tweets.
Started IO
Finished IO
Saved 101000 hydrated tweets.
Started IO
Finished IO
Saved 102000 hydrated tweets.
Started I

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (12,36) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


                                                full_text
0       @Belagavi_infra @MoHFW_INDIA There is no use o...
1       RT @dezying: With Occam's Razor, the plausible...
2       @CTVNews You mean the experimental vaccine we ...
3       RT @LukeGoslingMP: It is way past time that PM...
4       RT @SlimJosa: I don’t think some of you unders...
...                                                   ...
140049  RT @JandTContent: .@GovernorLittle @GovHawaii ...
140050  COVID-19 Vaccination appointments available at...
140051  RT @sridhar1085: The centre told the Supreme C...
140052  RT @FaerieWhings: Cuba did it again, with no p...
140053  RT @TwitterMoments: The Delta variant of COVID...

[140054 rows x 1 columns]
  Cloning https://github.com/stanfordnlp/stanza.git to /tmp/pip-req-build-gc8p1ujw
  Running command git clone -q https://github.com/stanfordnlp/stanza.git /tmp/pip-req-build-gc8p1ujw
  Created wheel for stanza: filename=stanza-1.2.1-cp37-none-any.whl size=642799 sha256=bc3269

INFO:stanza:Downloading default packages for language: en (English)...


INFO:stanza:Finished downloading models and saved to /root/stanza_resources.
INFO:stanza:Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | combined  |
| pos       | combined  |
| lemma     | combined  |
| depparse  | combined  |
| sentiment | sstplus   |
| ner       | ontonotes |

INFO:stanza:Use device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: depparse
INFO:stanza:Loading: sentiment
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:652: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool1d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [ ]:
#Código para conexão a pesquisa de tweets.
!pip install tweepy

In [ ]:
import tweepy
import webbrowser
import time

In [ ]:
consumer_key = "TqOMbK44mhFn8P4Rviae3sM5Y"
consumer_secret = "Qkog3ytEszYMQ531yPfjXupPNPOoh973v1YP1Oe6YZYWNY19Ba"
callback_uri = 'oob'

In [ ]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret, callback_uri)
redirect_url = auth.get_authorization_url()
print(redirect_url)

In [ ]:
user_pin_input = input("Digite o pin aqui: ")

In [ ]:
auth.get_access_token(user_pin_input)

In [ ]:
print(auth.access_token, auth.access_token_secret)

In [ ]:
api = tweepy.API(auth)

In [ ]:
me = api.me()

In [ ]:
print(me.screen_name)

In [ ]:
print(api.get_status(id = '1224527382494662659'))

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Thu Jun 17 14:20:10 2021

@author: jfrancis
"""
!pip install google_trans_new
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
import re
import tweepy
from tweepy import OAuthHandler
from textblob import TextBlob
from google_trans_new import google_translator
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet


class TwitterClient(object):
    '''
    Classe genérica do Twitter para analise de sentimentos
    '''
    def __init__(self):
        '''
        metodo de inicialização 
        '''
        # keys e tokens geradas no  dev console com meu usuario projeto twitter Unicamp_ml
        consumer_key = 'e9RRxn8rGRFijTZfG73VtdniU'
        consumer_secret = 'wvW8f7S0C60MMyCwGgpr2y3ypc0VhWXJsDb8edxiWqArtp1DuH'
        access_token = '1405577872413188096-dsPwQO9T8diH0E0xm2NaGccl0pjDqa'
        access_token_secret = 'f4huEGvJr9vZsX9t2uZRchYBrJul9iNuRUqY8CGMRO2mA'

        # tentando autenticação
        try:
            # criação do OAuthHandler object
            self.auth = OAuthHandler(consumer_key, consumer_secret)
            # set access token e secret
            self.auth.set_access_token(access_token, access_token_secret)
            # criando o tweepy API object para coletar os tweets
            self.api = tweepy.API(self.auth)
        except:
            print("Erro: Falha na autenticação")

    

    def get_tweets(self, query, count):
        '''
        Função principal para coletar os tweets e analisá-los
        '''
        # cria uma lista vazia para armazenar os tweets analizados
        # talvez aqui poderiamos definir o pandas ou dataset para armazenar
        # como o exemplo é simples, sem as funções de mineração então uma lista serve 
        tweets = []

        try:
            # chama a twitter api para coletar os tweets
            fetched_tweets = self.api.search(q = query, count = count)

            # analisa os tweets um a um
            for tweet in fetched_tweets:
                # cria um dicionario para armazenar os parametros requeridos de um tweet por exemplo 
                # parametro text, parametro sentiment
                parsed_tweet = {}

                # salva o texto do tweet
                parsed_tweet['text'] = tweet.text
                # salva o sentimento do tweet
                parsed_tweet['sentiment'] = self.get_tweet_sentiment(tweet.text)

                # inclui o tweet analisado na lista de tweets
                if tweet.retweet_count > 0:
                    # se o tweet tem retweets garante que vai ser incluido na lista uma unica vez
                    if parsed_tweet not in tweets:
                        tweets.append(parsed_tweet)
                else:
                    tweets.append(parsed_tweet)

            # retorna os tweets analisados
            return tweets

        except tweepy.TweepError as e:
            # imprime os erros se existirem
            print("Erro : " + str(e))

# cria o objeto do TwitterClient Class
api = TwitterClient()
# chama a função para coletar os tweets
tweets = api.get_tweets(query = 'covid vacinação', count = 200)





import numpy as np
my_array = np.array(tweets)
print (my_array)

In [ ]:
!pip install -U -q PyDrive
  
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
  
  
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
link = 'https://drive.google.com/file/d/16OgX2Hm5w2GNRvjb-iyfJHc52JJ4-E2v/view?usp=sharing'
  
import pandas as pd
  
# to get the id part of the file
id = link.split("/")[-2]
  
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('1_corona_tweets_463.csv')  
  
df = pd.read_csv('1_corona_tweets_463.csv')
df